### 1 player convo

In [ ]:
import json
from rich.console import Console
from rich.table import Table

file_path = '../'
file_path += 'outputs/2024-08-27/17-16-32/iteration_01/player_0_iter_01_game_0001.json'

table_name = 'player_example'
table_name += '.html'
to_html = True


with open(file_path, 'r') as f:
    conversation = json.load(f)

no_rules = True

# Initialize the console
console = Console(record=to_html)

# Create a table to display the conversation
table = Table(show_header=True, header_style="bold magenta")
table.add_column("Role", style="dim", width=12)
table.add_column("Content" )


for message in conversation:
    match message["role"]:
      case 'assistant':
        style = "blue"
      case _:
        style = "green"
    if message["role"] == 'user':
      table.add_row('Moderator', message["content"], style = style)
    else:
      table.add_row('Player', message["content"], style = style)
    table.add_row("", "")
table.add_section()

console.print(table)
if to_html:
  html = console.export_html()

  custom_css = """
  <style>
      body {
          font-size: 12px;
      }
  </style>
  """

  html = html.replace("<head>", f"<head>{custom_css}")

  with open(table_name, "w") as f:
      f.write(html)

### 2 players convo

In [ ]:
from itertools import zip_longest
import json
import heapq
from rich.console import Console
from rich.table import Table

iter = '01'
game = '0001'
file_path = '../'
file_path += 'outputs/2024-08-27/17-33-16'

player0_path = file_path + f'/iteration_{iter}/player_0_iter_{iter}_game_{game}.json'
player1_path = file_path + f'/iteration_{iter}/player_1_iter_{iter}_game_{game}.json'

table_name = 'aggrement_example'
table_name += '.html'
to_html = True


with open(player0_path, 'r') as f:
    convo_player_0 = [j['content'] for j in json.load(f)]

with open(player1_path, 'r') as f:
    convo_player_1 = [j['content'] for j in json.load(f)]

def is_moderator(item):
    return item.startswith('The other player') or item.startswith('A proposal has been made')

def filter_elements(lst):
    lst = lst[1:]
    last_element = lst[-1]
    result = []
    for id, item in enumerate(lst):
        if is_moderator(item):
            result.append(lst[id-1])
    
    result.append(last_element)
    return result

def extract_content(tag, text):
    start_tag = f"<{tag}>"
    end_tag = f"</{tag}>"
    if start_tag not in text:
        return ""
    start_index = text.find(start_tag) + len(start_tag)
    end_index = text.find(end_tag)
    return text[start_index:end_index].strip()

# Filtered lists
filtered_l1 = filter_elements(convo_player_0)
filtered_l2 = filter_elements(convo_player_1)

# Combine the lists
convo = [item for pair in zip_longest(filtered_l1, filtered_l2) for item in pair if item is not None]
# convo = []
# for a, c in zip_longest(filtered_l1, filtered_l2):
#     convo.extend([a, c])

# convo = list(heapq.merge(filtered_l1, filtered_l2))

# Initialize the console
console = Console(record=to_html)

# Create a table to display the conversation
table = Table(show_header=True, header_style="bold magenta")
table.add_column("Role", style="dim", width=12)
table.add_column("Content" )


table.add_row('Moderator', convo_player_0[0], style = "green")
table.add_row("", "")
for id, content in enumerate(convo):
    reasoning = extract_content("reason", content)
    message = extract_content("message", content) or extract_content("finalize", content)
    if id % 2 == 0:
      table.add_row('Player 0', reasoning, style = "dim blue")
      table.add_row("", "")
      table.add_row('', message, style = "blue")
    else:
      table.add_row('Player 1', reasoning, style = "dim red")
      table.add_row("", "")
      table.add_row('', message, style = "red")
    table.add_row("", "")
table.add_section()

console.print(table)

if to_html:
    html = console.export_html()

    custom_css = """
    <style>
        body {
            font-size: 12px;
        }
    </style>
    """

    html = html.replace("<head>", f"<head>{custom_css}")

    with open(table_name, "w") as f:
        f.write(html)
